In [1]:
library(lme4)
library(afex)
library(emmeans)
library(dplyr)
library(ggplot2)
library(svglite)
library(lmtest)
library(tidyr)
library(MCMCglmm)
library(reshape)
library(gridExtra)
library(cowplot)
library(svglite)
library(data.table)

Loading required package: Matrix

************
Welcome to afex. For support visit: http://afex.singmann.science/

- Functions for ANOVAs: aov_car(), aov_ez(), and aov_4()
- Methods for calculating p-values with mixed(): 'S', 'KR', 'LRT', and 'PB'
- 'afex_aov' and 'mixed' objects can be passed to emmeans() for follow-up tests
- Get and set global package options with: afex_options()
- Set sum-to-zero contrasts globally: set_sum_contrasts()
- For example analyses see: browseVignettes("afex")
************


Attaching package: ‘afex’


The following object is masked from ‘package:lme4’:

    lmer



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘tidyr’


The following objects 

In [2]:
LickITI <- read.csv('/home/bagotlab/eshaan.i/PRL/FP_PRL_Nov_2021/Code/Longform Data/PRL_ZScore_Licks.csv')

LickITI = subset(LickITI, (lp > 0))
LickITI$time_idx <- as.factor(as.numeric(as.factor(LickITI$time)))
LickITI<-LickITI[,c('index','time_idx','time','ID','rewards','lp','Day','rt','PFC','vHIP')]
LickITI<- LickITI %>% group_by(ID,Day) %>% mutate(Previous.Reward = lag(rewards,n=240))
LickITI<- LickITI %>% group_by(ID,Day) %>% mutate(Previous.Previous.Reward = lag(Previous.Reward,n=240))
LickITI<- LickITI %>% group_by(ID,Day) %>% mutate(Next.Choice = lead(lp,n=240))
LickITI<- LickITI %>% group_by(ID,Day) %>% mutate(Next.RT = lead(rt,n=240))
LickITI<- LickITI %>% group_by(ID,Day) %>% mutate(Previous.Choice = lag(lp,n=240))
LickITI$Stay <- ifelse(LickITI$lp == LickITI$Previous.Choice,1,0)
LickITI$StayNext <- ifelse(LickITI$lp == LickITI$Next.Choice,1,0)
LickITI$sex <- as.factor(ifelse(LickITI$ID > 212,"Female", "Male"))

LickDF<- as.data.frame(LickITI[((LickITI$time)>=0)&((LickITI$time)<=2),] %>% 
                group_by(index)%>%summarize(PFC=mean(PFC),vHIP=mean(vHIP),rewards=mean(rewards),ID=mean(ID),Previous.Reward=mean(Previous.Reward), Previous.Previous.Reward=mean(Previous.Previous.Reward),Stay=mean(Stay), StayNext=mean(StayNext), lp=mean(lp),Day=mean(Day)))

LickDF$ID<-as.factor(LickDF$ID)
LickDF$Day<-as.factor(LickDF$Day)
LickDF$rewards <- as.factor(ifelse(LickDF$rewards == 0,"Unrewarded", "Rewarded"))
LickDF$Previous.Reward <- as.factor(ifelse(LickDF$Previous.Reward == 0,"Unrewarded", "Rewarded"))
LickDF$Stay <- as.factor(ifelse(LickDF$Stay == 0,"Stay", "Shift"))
LickDF$lp <- as.factor(ifelse(LickDF$lp == 1,"Contralateral", "Ipsilateral"))

LickITI$rewards<-as.factor(LickITI$rewards)
LickITI$ID<-as.factor(LickITI$ID)
LickITI$Day<-as.factor(LickITI$Day)
LickITI$Previous.Reward <-as.factor(LickITI$Previous.Reward)
LickITI$rewards <- as.factor(ifelse(LickITI$rewards == 0,"Unrewarded", "Rewarded"))
LickITI$Stay <- as.factor(ifelse(LickITI$Stay == 0,"Shift", "Stay"))
LickITI$lp <- as.factor(ifelse(LickITI$lp == 1,"Contralateral", "Ipsilateral"))


In [4]:
plot_theme <- theme(axis.title=element_text(size=25,color='gray30'),
                    axis.line=element_line(color='gray30'),
                    axis.text.y=element_text(size=20,color='gray30'),
                    axis.text.x=element_text(size=20,color='gray30'),
                    legend.text=element_text(size=20),
                    legend.key.size=unit(2,'mm'),
                    panel.background=element_blank(),
                    panel.grid.major.y = element_blank(),
                    legend.position = 'none',
                    plot.margin = margin(0.5,0.5,0.5,0.5,'cm'),
                    strip.background = element_blank(),
                    strip.text = element_text(size=20,color='gray30')) 

plot_theme_PFC <- list(plot_theme, 
                        scale_color_manual(values=c('Rewarded'="#B51D1D",  'Unrewarded'="#444444",'Stay'="#791bfa", 'Shift'="#ad74fb", "Contralateral"="#6ad53a", "Ipsilateral"="#246706")),
                        scale_fill_manual(values=c('Rewarded'="#B51D1D",  'Unrewarded'="#444444", 'Stay'="#791bfa", 'Shift'="#ad74fb","Contralateral"="#6ad53a", "Ipsilateral"="#246706")),
                        scale_shape_manual(values=c('Male'=16,'Female'=17)))
plot_theme_vHIP <- list(plot_theme, 
                        scale_color_manual(values=c('Rewarded'="#F27E19", 'Unrewarded'="#444444",'Stay'="#000bfa", 'Shift'="#787ef9", "Contralateral"="#f1c232", "Ipsilateral"="#bc9a35")), 
                        scale_fill_manual(values=c('Rewarded'="#F27E19", 'Unrewarded'="#444444",'Stay'="#000bfa", 'Shift'="#787ef9","Contralateral"="#f1c232", "Ipsilateral"="#bc9a35")),
                        scale_shape_manual(values=c('Male'=16,'Female'=17)))

In [ ]:
#Figure A

PFC1BackLicks = NULL
for (tm in 1:240){
myModel=lmer(PFC ~ rewards*sex + (1|ID)+(1|Day),data=subset(LickITI, (time_idx==tm)))
current_time = mean(subset(LickITI, (time_idx==tm))$time)
PFC1BackLicks= rbind(PFC1BackLicks,cbind(time=current_time,as.data.frame(emmeans(myModel, ~ rewards|sex))))
}

pfclicks<-ggplot(data = PFC1BackLicks)+
  geom_line(aes(time,emmean,group=rewards, colour=rewards))+
  geom_ribbon(aes(x=time,y = emmean, ymin = emmean - SE, ymax = emmean + SE, fill = rewards), alpha = .2)+
  xlab('Time from Lick')+
  ylab('Mean Z-Score')+
  geom_vline(xintercept=0, linetype="dotted", size = 1)+
  plot_theme_PFC+
  facet_grid(~sex)

In [ ]:
#Figure D

vHIP1BackLicks = NULL
for (tm in 1:240){
myModel=lmer(vHIP ~ rewards*sex + (1|ID)+(1|Day),data=subset(LickITI, (time_idx==tm)))
current_time = mean(subset(LickITI, (time_idx==tm))$time)
vHIP1BackLicks= rbind(vHIP1BackLicks,cbind(time=current_time,as.data.frame(emmeans(myModel, ~ rewards|sex))))
}

vhiplicks<-ggplot(data = vHIP1BackLicks)+
  geom_line(aes(time,emmean,group=rewards, colour=rewards))+
  geom_ribbon(aes(x=time,y = emmean, ymin = emmean - SE, ymax = emmean + SE, fill = rewards), alpha = .2)+
  xlab('Time from Lick')+
  ylab('Mean Z-Score')+
  geom_vline(xintercept=0, linetype="dotted", size = 1)+
  plot_theme_vHIP +
  facet_grid (~sex)

In [3]:
ITI <- read.csv('/home/bagotlab/eshaan.i/PRL/FP_PRL_Nov_2021/Code/Longform Data/CondensedPRL_ZScore_ITI.csv')
ITI = subset(ITI,ID!=223)
ITI$time_idx <- as.factor(as.numeric(as.factor(ITI$time)))
ITI<-ITI[,c('index','time_idx','time','ID','rewards','lp','Day','rt','PFC','vHIP')]
ITI$sex <- as.factor(ifelse(ITI$ID > 212,"Female", "Male"))
ITI$Day <- as.numeric(as.factor(ITI$Day))

Behavior<- ITI %>% group_by(index)%>%summarize(rewards=mean(rewards),ID=mean(ID),rt=mean(rt),lp=mean(lp), Day=mean(Day))
Behavior$TrialType <- as.factor(ifelse(Behavior$rewards==1, 'Rewards', ifelse(Behavior$lp==0, 'Omission', 'Unrewarded')))
Behavior$sex <- as.factor(ifelse(Behavior$ID > 212,"Female", "Male"))

ITI = subset(ITI, lp > 0)
ITI$time_idx <- as.factor(as.numeric(as.factor(ITI$time)))
ITI<- ITI %>% group_by(ID,Day) %>% mutate(Previous.Reward = lag(rewards,n=240))
ITI<- ITI %>% group_by(ID,Day) %>% mutate(Previous.Previous.Reward = lag(Previous.Reward,n=240))
ITI<- ITI %>% group_by(ID,Day) %>% mutate(Next.Choice = lead(lp,n=240))
ITI<- ITI %>% group_by(ID,Day) %>% mutate(Next.RT = lead(rt,n=240))
ITI<- ITI %>% group_by(ID,Day) %>% mutate(Previous.Choice = lag(lp,n=240))
ITI$Stay <- ifelse(ITI$lp == ITI$Previous.Choice,1,0)
ITI$StayNext <- ifelse(ITI$lp == ITI$Next.Choice,1,0)

enditi<- as.data.frame(ITI[((ITI$time)>=8)&((ITI$time)<=10),] %>% 
                group_by(index)%>%summarize(PFC=mean(PFC),vHIP=mean(vHIP),rewards=mean(rewards),ID=mean(ID),Previous.Reward=mean(Previous.Reward), Previous.Previous.Reward=mean(Previous.Previous.Reward),Stay=mean(Stay), StayNext=mean(StayNext), lp=mean(lp),Day=mean(Day), PFCend=mean(PFC),vHIPend=mean(vHIP)))
Post_DF <- as.data.frame(ITI[((ITI$time)>=0)&((ITI$time)<=2),] %>% group_by(index)%>%summarize(PFC_Post=mean(PFC),vHIP_Post=mean(vHIP)))
Pre_DF <- as.data.frame(ITI[((ITI$time)>=-2)&((ITI$time)<0),] %>% group_by(index)%>%summarize(PFC_Pre=mean(PFC),vHIP_Pre=mean(vHIP)))

DF <- merge(enditi, Post_DF)
DF <- merge(DF, Pre_DF)
DF$sex <- as.factor(ifelse(DF$ID > 212,"Female", "Male"))
DF$ID<-as.factor(DF$ID)
DF$Day<-as.factor(DF$Day)
DF$rewards <- as.factor(ifelse(DF$rewards == 0,"Unrewarded", "Rewarded"))
DF$Previous.Reward <- as.factor(ifelse(DF$Previous.Reward == 0,"Unrewarded", "Rewarded"))
DF$Stay <- as.factor(ifelse(DF$Stay == 0,"Shift", "Stay"))
DF$lp <- as.factor(ifelse(DF$lp == 1,"Contralateral", "Ipsilateral"))

ITI$rewards<-as.factor(ITI$rewards)
ITI$ID<-as.factor(ITI$ID)
ITI$Day<-as.factor(ITI$Day)
ITI$Previous.Reward <-as.factor(ITI$Previous.Reward)
ITI$rewards <- as.factor(ifelse(ITI$rewards == 0,"Unrewarded", "Rewarded"))
ITI$Stay <- as.factor(ifelse(ITI$Stay == 0,"Shift", "Stay"))
ITI$lp <- as.factor(ifelse(ITI$lp == 1,"Contralateral", "Ipsilateral"))


In [ ]:
#Figure B

PFC1BackStay = NULL
for (tm in 1:80){
myModel=lmer(PFC ~ Stay*sex +(1|Previous.Reward)+(1|ID)+(1|Day),data=subset(ITI, (time_idx==tm)))
current_time = mean(subset(ITI, (time_idx==tm))$time)
PFC1BackStay= rbind(PFC1BackStay,cbind(time=current_time,as.data.frame(emmeans(myModel, ~ Stay|sex))))
}

ggplot(data = PFC1BackStay)+
  geom_line(aes(time,emmean,group=Stay, colour=Stay))+
  geom_ribbon(aes(x=time,y = emmean, ymin = emmean - SE, ymax = emmean + SE, fill = Stay), alpha = .2)+
  xlab('Time from LP')+
  ylab('mPFC Activity')+
  geom_vline(xintercept=0, linetype="dotted", size = 1)+
  plot_theme_PFC+
  facet_grid(~sex)

In [ ]:
#Figure E

vHIP1BackStay = NULL
for (tm in 1:80){
myModel=lmer(vHIP ~ Stay*sex +(1|Previous.Reward)+(1|ID)+(1|Day),data=subset(ITI, (time_idx==tm)))
current_time = mean(subset(ITI, (time_idx==tm))$time)
vHIP1BackStay= rbind(vHIP1BackStay,cbind(time=current_time,as.data.frame(emmeans(myModel, ~ Stay|sex))))
}

ggplot(data = vHIP1BackStay)+
  geom_line(aes(time,emmean,group=Stay, colour=Stay))+
  geom_ribbon(aes(x=time,y = emmean, ymin = emmean - SE, ymax = emmean + SE, fill = Stay), alpha = .2)+
  xlab('Time from LP')+
  ylab('vHip Activity')+
  plot_theme_vHIP + 
  geom_vline(xintercept=0, linetype="dotted", size = 1)+
  facet_grid(~sex)

In [ ]:
#Figure C

PFC1Backlp = NULL
for (tm in 1:80){
myModel=lmer(PFC ~ lp*sex +(1|Previous.Reward)+(1|ID)+(1|Day),data=subset(ITI, (time_idx==tm)))
current_time = mean(subset(ITI, (time_idx==tm))$time)
PFC1Backlp= rbind(PFC1Backlp,cbind(time=current_time,as.data.frame(emmeans(myModel, ~ lp|sex))))
}

ggplot(data = PFC1Backlp)+
  geom_line(aes(time,emmean,group=lp, colour=lp))+
  geom_ribbon(aes(x=time,y = emmean, ymin = emmean - SE, ymax = emmean + SE, fill = lp), alpha = .2)+
  xlab('Time from LP')+
  ylab('mPFC Activity')+
  geom_vline(xintercept=0, linetype="dotted", size = 1)+
  plot_theme_PFC+
  facet_grid(~sex)

In [ ]:
#Figure F

vHIP1Backlp = NULL
for (tm in 1:80){
myModel=lmer(vHIP ~ lp*sex +(1|Previous.Reward)+(1|ID)+(1|Day),data=subset(ITI, (time_idx==tm)))
current_time = mean(subset(ITI, (time_idx==tm))$time)
vHIP1Backlp= rbind(vHIP1Backlp,cbind(time=current_time,as.data.frame(emmeans(myModel, ~ lp|sex))))
}

ggplot(data = vHIP1Backlp)+
  geom_line(aes(time,emmean,group=lp, colour=lp))+
  geom_ribbon(aes(x=time,y = emmean, ymin = emmean - SE, ymax = emmean + SE, fill = lp), alpha = .2)+
  xlab('Time from LP')+
  ylab('vHip Activity')+
  geom_vline(xintercept=0, linetype="dotted", size = 1)+
  plot_theme_vHIP+
  facet_grid(~sex)